In [2]:
import requests
import pandas as pd
import time

import warnings
warnings.filterwarnings('ignore')

In [3]:
api_key = 'ENTER NEOPLE API'

In [4]:
# 아이템 정보 가져오기 API
def df_search_item(itemName = '', item_hashtag = ''):
    url = 'https://api.neople.co.kr/df/items'

    params = {
        'limit' : 30,
        'apikey' : api_key,
        'wordType' : 'match'
    }

    if itemName != '':
        params['itemName'] = itemName
    if item_hashtag != '':
        params['hashtag'] = item_hashtag

    return requests.get(url, params=params)

# 아이템 세부정보 가져오기 API
def df_get_item_details(itemId):
    url = f'https://api.neople.co.kr/df/items/{itemId}?apikey={api_key}'
    return requests.get(url)
    
# 경매장 등록 아이템 정보 가져오기 API
def df_aution_item(itemName, limit=100):
    url = f'https://api.neople.co.kr/df/auction'
    params = {
        'itemName' : itemName,
        'limit' : limit,
        'sort' : 'auctionNo:desc',
        'apikey' : api_key
        
    }
    
    return requests.get(url, params=params)


# 경매장 판매 이력 정보 가져오기 API
def df_aution_sold(itemName, limit=100):
    url = f'https://api.neople.co.kr/df/auction-sold'
    params = {
        'itemName' : itemName,
        'limit' : limit,
        'apikey' : api_key
    }
    
    return requests.get(url, params = params)

In [5]:
# 새로운 아이템인지 확인
def check_new_items(check_cols, new_check_lst, isFirst, check_type, itemName = ''):

    new_items = []
    
    if check_type == 'regi':

        if isFirst:     # 계속 켜놓지 못하니까 처음에는 저장된 DB에서 데이터를 불러와 확인하기 
            print('Frist Search in REGI')
            auctionIds = get_data_from_db(check_type) #DB에서 거래 타입으로 테이블에서 정보 가져오기

            for check_value in check_cols: #새로운 아이템을 하나씩 검사
                if str(check_value) not in auctionIds: # DB에 있는 옥션ID인지 확인하고 없으면 새로운 아이템에추가, 아니면 체크 완료 리스트에만 추가
                    new_items.append(check_value)
                new_check_lst.append(check_value)   

        
        else:
            for check_value in check_cols:
                if check_value not in new_check_lst:
                    new_items.append(check_value)
                    new_check_lst.append(check_value)
                
    elif check_type == 'sold':
        if isFirst:
            print('Frist Search in SOLD')
            db_sold_dates = get_data_from_db(check_type, target = itemName) # DB에서 입장권 이름과 거래 타입에 맞는 데이터만 가져옴
            
            for check_value in check_cols:
                if check_value not in db_sold_dates: # DB에 있는 거래일시인지 확인하고 없다면 새로운 아이템에 추가, 아니면 체크 완료 리스트에만 추가
                    new_items.append(check_value)
                new_check_lst[itemName].append(check_value)   # 체크 완료 리스트는 아이템명별로 거래일시 정보를 그룹화

        else:
            for check_value in check_cols:
                if check_value not in new_check_lst[itemName] :
                    new_items.append(check_value)
                    new_check_lst[itemName].append(check_value)
                
    return new_items


# DB에 데이터를 등록하기 위해 데이터프레임음 만드는 함수
def make_insert_item(df, check_type):
    
    if check_type == 'regi':
        use_cols = ['auctionNo', 'regDate', 'itemId', 'itemName', 'count', 'unitPrice']
        
    elif check_type == 'sold':
        use_cols = ['soldDate', 'itemId', 'itemName', 'count', 'unitPrice']
        
    
    excecuteDB(df[use_cols], check_type)    # 경매장 등록 함수 호출
    

# 경매장에서 아이템 검색 함수 호출
def searching_item(target, check_type):
    
    if check_type == 'regi':
        return pd.DataFrame(df_aution_item(target).json()['rows']) 
    
    elif check_type == 'sold':
        return pd.DataFrame(df_aution_sold(target).json()['rows']) 


# 경매장에서 새로운 아이템을 탐색하고 등록하는 메인 함수
# 이 함수를 호출하면 경매장에서 아이템을 검색하고 새로운 아이템인지 확인하고 DB에 등록
def search_and_regi_item(target, new_check_regi_lst, new_check_sold_lst, isFrist, check_type):
    
    df = searching_item(target, check_type)
    
    if len(df) == 0:
#         print('No Data')
        return 
    
    # 경매장 등록 새로운 아이템 체크
    if check_type == 'regi':
        new_item = df['auctionNo']
        new_item = check_new_items(new_item, new_check_regi_lst, isFrist, check_type)
        df = df[df['auctionNo'].isin(new_item)] # 새롭게 등록된 아이템 필터링

    # 경매장 판매 새로운 아이템 체크
    elif check_type == 'sold':
        new_item = df['soldDate']
        new_item = check_new_items(new_item, new_check_sold_lst, isFrist, check_type, itemName = target)
        df = df[df['soldDate'].isin(new_item)] # 새롭게 등록된 아이템 필터링
        
    if len(df) > 0:  # 새로운 아이템이 있다면 몇 개 있는지 출력
        print(f'{check_type} {target} New {len(new_item)} items')  
        
    make_insert_item(df, check_type) # DB등록

In [6]:
import  mysql.connector

# DB 연결 함수
def init_mysql():
    aws = mysql.connector.Connect(
        database = 'df',
        host = "dungeonandfighter-db.crnv1am9ifjd.ap-northeast-2.rds.amazonaws.com",
        port = 3306,
        user = 'sold_table',
        password = '1234'
    )
    return aws

In [7]:
# DB에서 데이터 가져오기
def get_data_from_db(check_type, target = ''):
    aws = init_mysql()
    cur = aws.cursor()
    
    if check_type == 'regi':
        query = 'SELECT auctionNo FROM ticket_history'
        cur.execute(query)
        result = pd.DataFrame(cur.fetchall(), columns={'auctionNo'})['auctionNo'].values
        aws.close()
        return result
    
    elif check_type == 'sold':
        query = f"Select soldDate from sold_history where itemName like '%{target}%'"
        cur.execute(query)
        result = pd.DataFrame(cur.fetchall(), columns={'soldDate'})['soldDate'].values
        aws.close()
        return result

In [8]:
# DB에 데이터 삽입
def excecuteDB(df, check_type):
    aws = init_mysql()
    cur = aws.cursor()
    cols = df.columns
    insert_cols = ",".join(cols)
    
    for idx, row in df.iterrows():
        insert_value = ['"'+str(row[col])+'"' for col in cols]
        insert_value = ",".join(map(str,insert_value))
        
        query = ''
        
        if check_type == 'regi':
            query = 'INSERT INTO ticket_history('+ insert_cols +') VALUES('+ insert_value +');'
            
        elif check_type == 'sold':
            query = 'INSERT INTO sold_history('+ insert_cols +') VALUES('+ insert_value +');'
        
        if query != '':
            cur.execute(query)
        
    aws.commit()
    aws.close()
    

In [9]:
# 티켓 이름 만들기 - API를 통해서 정보 가져오기
def make_ticket_names():
    high_dungeon = ['마이스터의 실험실', '파괴된 죽은 자의 성', '노블레스 코드', '노블레스 코드 : 더스크']
    grades = ['(노멀)' , '(익스퍼트)', '(마스터)']

    ticket_naems = [dungeon + " " + grade + ' 입장권' for grade in grades for dungeon in high_dungeon]
    ticket_naems

    item_dicts = dict()

    for name in ticket_naems:
        items = df_search_item(itemName=name).json()['rows']
        
        # 입장권의 획득 방식으로 동명인 아이템이 존재하기 때문에 구분짓기 위해 이름 뒤에 숫자를 붙임
        for idx, item in enumerate(items):
            item_dicts[item['itemName']+str(idx)] = item['itemId']

    return item_dicts


# 획득처 중 경매장을 통해서만 획득 가능한 아이템 정보 가져오기
def duplicated_dicts():
    item_dicts = make_ticket_names()
    result = dict()
    for key,item_id in item_dicts.items():
        if '경매장' in df_get_item_details(item_id).json()['itemObtainInfo']:

            result[key[:-1]] = item_id #itemName 맨뒤 숫자 제거
    
    return result


In [10]:
# 경매장에서 거래 가능한 아이템 정보 가져오기
item_dicts = duplicated_dicts()

In [11]:
item_dicts

{'마이스터의 실험실 (노멀) 입장권': '777bbcc882244a8f659008f08ea11b1d',
 '파괴된 죽은 자의 성 (노멀) 입장권': '0fffea6206e0d31debeb6702bfc39e38',
 '노블레스 코드 (노멀) 입장권': 'e840a926dfa1eaa57d8f109d0785283d',
 '노블레스 코드 : 더스크 (노멀) 입장권': '0dffb978eeed4ae827a53e70f6b5de33',
 '마이스터의 실험실 (익스퍼트) 입장권': '2b40a1d04038ba6ab707eef127498898',
 '파괴된 죽은 자의 성 (익스퍼트) 입장권': '2fb818c03e6475f4bb190ba82ebd4d76',
 '노블레스 코드 (익스퍼트) 입장권': 'e946558f1cfe9c315f8b5f57fb717ce7',
 '노블레스 코드 : 더스크 (익스퍼트) 입장권': 'ce0dea2ae74159f6c89c26367cbf0ae0',
 '마이스터의 실험실 (마스터) 입장권': 'c8ec3e7cd33ecfa87ab5266a11937b7a',
 '파괴된 죽은 자의 성 (마스터) 입장권': '746e21b9162cf7ea9548d0b2d44ea277',
 '노블레스 코드 (마스터) 입장권': '5df4eeed76ba82d8870a573dc0fea1b2',
 '노블레스 코드 : 더스크 (마스터) 입장권': 'ed016937130724ce812750cb25cb9ff8'}

In [ ]:
# 경매장 데이터 탐색
# 등록과 판매 체크리스트가 따로 존재.
# 판매의 경우 아이템별로 판매일시 정보가 들어가 있음.

new_check_regi_lst = []
new_check_sold_lst = {search_id : [] for search_id in item_dicts.keys()}
search_type = ['regi', 'sold']
isFrist = True

while True:
    for target in item_dicts.keys():
        try:
            for check_type in search_type:
                search_and_regi_item(target, new_check_regi_lst, new_check_sold_lst, isFrist, check_type = check_type)
        except Exception as e:
            print(f"{target} Error -> ", e)
            time.sleep(1)
            
    isFrist = False
    time.sleep(0.5)

Frist Search in REGI
Frist Search in SOLD
Frist Search in REGI
Frist Search in SOLD
Frist Search in REGI
Frist Search in SOLD
Frist Search in REGI
Frist Search in SOLD
Frist Search in REGI
Frist Search in SOLD
Frist Search in REGI
Frist Search in SOLD
Frist Search in REGI
Frist Search in SOLD
Frist Search in REGI
Frist Search in SOLD
Frist Search in REGI
Frist Search in SOLD
sold 마이스터의 실험실 (마스터) 입장권 New 5 items
Frist Search in REGI
Frist Search in SOLD
Frist Search in REGI
regi 노블레스 코드 (마스터) 입장권 New 1 items
Frist Search in SOLD
Frist Search in SOLD
regi 파괴된 죽은 자의 성 (노멀) 입장권 New 1 items
regi 파괴된 죽은 자의 성 (익스퍼트) 입장권 New 1 items
regi 파괴된 죽은 자의 성 (노멀) 입장권 New 1 items
regi 노블레스 코드 : 더스크 (노멀) 입장권 New 1 items
sold 마이스터의 실험실 (익스퍼트) 입장권 New 2 items
regi 파괴된 죽은 자의 성 (익스퍼트) 입장권 New 1 items
sold 노블레스 코드 : 더스크 (익스퍼트) 입장권 New 1 items
regi 노블레스 코드 : 더스크 (노멀) 입장권 New 1 items
regi 마이스터의 실험실 (노멀) 입장권 New 1 items
regi 파괴된 죽은 자의 성 (익스퍼트) 입장권 New 1 items
regi 마이스터의 실험실 (마스터) 입장권 New 1 items
regi 노블레스 코드 (마스

regi 파괴된 죽은 자의 성 (익스퍼트) 입장권 New 1 items
sold 마이스터의 실험실 (마스터) 입장권 New 1 items
sold 파괴된 죽은 자의 성 (노멀) 입장권 New 1 items
regi 마이스터의 실험실 (마스터) 입장권 New 1 items
sold 파괴된 죽은 자의 성 (마스터) 입장권 New 3 items
regi 파괴된 죽은 자의 성 (노멀) 입장권 New 1 items
regi 마이스터의 실험실 (마스터) 입장권 New 1 items
sold 마이스터의 실험실 (마스터) 입장권 New 1 items
regi 마이스터의 실험실 (마스터) 입장권 New 1 items
sold 파괴된 죽은 자의 성 (마스터) 입장권 New 2 items
regi 파괴된 죽은 자의 성 (익스퍼트) 입장권 New 1 items
regi 마이스터의 실험실 (마스터) 입장권 New 1 items
sold 노블레스 코드 : 더스크 (노멀) 입장권 New 1 items
regi 마이스터의 실험실 (마스터) 입장권 New 1 items
sold 노블레스 코드 : 더스크 (마스터) 입장권 New 1 items
regi 마이스터의 실험실 (익스퍼트) 입장권 New 1 items
sold 마이스터의 실험실 (마스터) 입장권 New 2 items
regi 마이스터의 실험실 (마스터) 입장권 New 3 items
sold 마이스터의 실험실 (익스퍼트) 입장권 New 1 items
regi 파괴된 죽은 자의 성 (익스퍼트) 입장권 New 1 items
regi 노블레스 코드 (익스퍼트) 입장권 New 1 items
regi 마이스터의 실험실 (마스터) 입장권 New 1 items
sold 마이스터의 실험실 (마스터) 입장권 New 1 items
regi 파괴된 죽은 자의 성 (노멀) 입장권 New 1 items
regi 마이스터의 실험실 (익스퍼트) 입장권 New 1 items
regi 파괴된 죽은 자의 성 (익스퍼트) 입장권 New 1 items
regi 노블레스 

노블레스 코드 (마스터) 입장권 Error ->  HTTPSConnectionPool(host='api.neople.co.kr', port=443): Max retries exceeded with url: /df/auction?itemName=%EB%85%B8%EB%B8%94%EB%A0%88%EC%8A%A4+%EC%BD%94%EB%93%9C+%28%EB%A7%88%EC%8A%A4%ED%84%B0%29+%EC%9E%85%EC%9E%A5%EA%B6%8C&limit=100&sort=auctionNo%3Adesc&apikey=Sxwiysbra37brnJK9Fp12Z81vXvU0IEN (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fd5f6fd2850>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))
노블레스 코드 : 더스크 (마스터) 입장권 Error ->  HTTPSConnectionPool(host='api.neople.co.kr', port=443): Max retries exceeded with url: /df/auction?itemName=%EB%85%B8%EB%B8%94%EB%A0%88%EC%8A%A4+%EC%BD%94%EB%93%9C+%3A+%EB%8D%94%EC%8A%A4%ED%81%AC+%28%EB%A7%88%EC%8A%A4%ED%84%B0%29+%EC%9E%85%EC%9E%A5%EA%B6%8C&limit=100&sort=auctionNo%3Adesc&apikey=Sxwiysbra37brnJK9Fp12Z81vXvU0IEN (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fd5f6fd2670>: Failed to establish a new 

regi 노블레스 코드 (익스퍼트) 입장권 New 1 items
sold 마이스터의 실험실 (마스터) 입장권 New 3 items
regi 마이스터의 실험실 (마스터) 입장권 New 1 items
sold 노블레스 코드 (익스퍼트) 입장권 New 1 items
regi 마이스터의 실험실 (마스터) 입장권 New 1 items
regi 노블레스 코드 (마스터) 입장권 New 1 items
regi 마이스터의 실험실 (마스터) 입장권 New 2 items
regi 노블레스 코드 (노멀) 입장권 New 1 items
regi 파괴된 죽은 자의 성 (마스터) 입장권 New 1 items
regi 파괴된 죽은 자의 성 (노멀) 입장권 New 1 items
sold 마이스터의 실험실 (마스터) 입장권 New 2 items
sold 파괴된 죽은 자의 성 (마스터) 입장권 New 2 items
sold 마이스터의 실험실 (마스터) 입장권 New 1 items
regi 마이스터의 실험실 (마스터) 입장권 New 1 items
regi 파괴된 죽은 자의 성 (익스퍼트) 입장권 New 1 items
regi 노블레스 코드 : 더스크 (익스퍼트) 입장권 New 1 items
regi 마이스터의 실험실 (마스터) 입장권 New 1 items
regi 파괴된 죽은 자의 성 (익스퍼트) 입장권 New 1 items
regi 마이스터의 실험실 (익스퍼트) 입장권 New 1 items
sold 파괴된 죽은 자의 성 (노멀) 입장권 New 3 items
sold 노블레스 코드 : 더스크 (익스퍼트) 입장권 New 1 items
sold 파괴된 죽은 자의 성 (마스터) 입장권 New 1 items
sold 마이스터의 실험실 (마스터) 입장권 New 5 items
regi 마이스터의 실험실 (마스터) 입장권 New 1 items
sold 마이스터의 실험실 (마스터) 입장권 New 1 items
regi 파괴된 죽은 자의 성 (마스터) 입장권 New 1 items
sold 마이스터의 실험실 (마스

sold 마이스터의 실험실 (마스터) 입장권 New 3 items
regi 노블레스 코드 (마스터) 입장권 New 1 items
regi 파괴된 죽은 자의 성 (익스퍼트) 입장권 New 1 items
regi 파괴된 죽은 자의 성 (마스터) 입장권 New 1 items
regi 노블레스 코드 : 더스크 (마스터) 입장권 New 1 items
regi 파괴된 죽은 자의 성 (익스퍼트) 입장권 New 1 items
regi 노블레스 코드 (익스퍼트) 입장권 New 1 items
regi 마이스터의 실험실 (익스퍼트) 입장권 New 1 items
sold 마이스터의 실험실 (마스터) 입장권 New 1 items
regi 파괴된 죽은 자의 성 (마스터) 입장권 New 1 items
regi 마이스터의 실험실 (마스터) 입장권 New 1 items
sold 파괴된 죽은 자의 성 (노멀) 입장권 New 1 items
sold 노블레스 코드 : 더스크 (마스터) 입장권 New 1 items
regi 노블레스 코드 (익스퍼트) 입장권 New 1 items
sold 노블레스 코드 : 더스크 (익스퍼트) 입장권 New 3 items
sold 마이스터의 실험실 (마스터) 입장권 New 1 items
sold 파괴된 죽은 자의 성 (익스퍼트) 입장권 New 3 items
regi 파괴된 죽은 자의 성 (노멀) 입장권 New 1 items
sold 파괴된 죽은 자의 성 (익스퍼트) 입장권 New 2 items
sold 마이스터의 실험실 (마스터) 입장권 New 1 items
sold 파괴된 죽은 자의 성 (노멀) 입장권 New 2 items
regi 노블레스 코드 : 더스크 (익스퍼트) 입장권 New 1 items
regi 노블레스 코드 : 더스크 (마스터) 입장권 New 2 items
regi 노블레스 코드 (마스터) 입장권 New 1 items
regi 파괴된 죽은 자의 성 (익스퍼트) 입장권 New 1 items
sold 파괴된 죽은 자의 성 (마스터) 입장권 New 1 ite

regi 노블레스 코드 : 더스크 (노멀) 입장권 New 1 items
regi 마이스터의 실험실 (마스터) 입장권 New 1 items
sold 마이스터의 실험실 (마스터) 입장권 New 3 items
sold 마이스터의 실험실 (마스터) 입장권 New 2 items
sold 노블레스 코드 (익스퍼트) 입장권 New 1 items
sold 파괴된 죽은 자의 성 (마스터) 입장권 New 1 items
regi 노블레스 코드 (마스터) 입장권 New 1 items
sold 마이스터의 실험실 (마스터) 입장권 New 2 items
sold 노블레스 코드 (마스터) 입장권 New 1 items
regi 노블레스 코드 : 더스크 (마스터) 입장권 New 1 items
regi 마이스터의 실험실 (익스퍼트) 입장권 New 1 items
sold 노블레스 코드 (익스퍼트) 입장권 New 1 items
regi 노블레스 코드 (노멀) 입장권 New 1 items
regi 마이스터의 실험실 (마스터) 입장권 New 1 items
sold 마이스터의 실험실 (마스터) 입장권 New 1 items
regi 파괴된 죽은 자의 성 (익스퍼트) 입장권 New 1 items
sold 마이스터의 실험실 (마스터) 입장권 New 1 items
regi 노블레스 코드 (마스터) 입장권 New 1 items
regi 파괴된 죽은 자의 성 (노멀) 입장권 New 1 items
regi 마이스터의 실험실 (익스퍼트) 입장권 New 1 items
sold 마이스터의 실험실 (마스터) 입장권 New 3 items
sold 노블레스 코드 (마스터) 입장권 New 1 items
regi 노블레스 코드 (익스퍼트) 입장권 New 1 items
sold 마이스터의 실험실 (마스터) 입장권 New 1 items
sold 노블레스 코드 : 더스크 (마스터) 입장권 New 1 items
regi 파괴된 죽은 자의 성 (노멀) 입장권 New 1 items
regi 노블레스 코드 : 더스크 (노멀) 입장권 New 

sold 노블레스 코드 (마스터) 입장권 New 1 items
regi 노블레스 코드 : 더스크 (익스퍼트) 입장권 New 1 items
sold 마이스터의 실험실 (마스터) 입장권 New 1 items
sold 노블레스 코드 : 더스크 (노멀) 입장권 New 1 items
regi 노블레스 코드 : 더스크 (익스퍼트) 입장권 New 1 items
regi 마이스터의 실험실 (마스터) 입장권 New 1 items
sold 마이스터의 실험실 (마스터) 입장권 New 1 items
regi 마이스터의 실험실 (마스터) 입장권 New 2 items
regi 파괴된 죽은 자의 성 (익스퍼트) 입장권 New 1 items
regi 노블레스 코드 (익스퍼트) 입장권 New 1 items
regi 마이스터의 실험실 (마스터) 입장권 New 2 items
sold 마이스터의 실험실 (마스터) 입장권 New 3 items
sold 노블레스 코드 (익스퍼트) 입장권 New 1 items
sold 노블레스 코드 : 더스크 (익스퍼트) 입장권 New 1 items
regi 마이스터의 실험실 (마스터) 입장권 New 2 items
regi 마이스터의 실험실 (마스터) 입장권 New 1 items
sold 마이스터의 실험실 (마스터) 입장권 New 2 items
regi 마이스터의 실험실 (마스터) 입장권 New 1 items
regi 노블레스 코드 : 더스크 (노멀) 입장권 New 1 items
sold 파괴된 죽은 자의 성 (익스퍼트) 입장권 New 1 items
regi 노블레스 코드 : 더스크 (익스퍼트) 입장권 New 1 items
sold 노블레스 코드 : 더스크 (익스퍼트) 입장권 New 1 items
regi 마이스터의 실험실 (익스퍼트) 입장권 New 1 items
regi 파괴된 죽은 자의 성 (익스퍼트) 입장권 New 1 items
sold 노블레스 코드 : 더스크 (익스퍼트) 입장권 New 1 items
sold 마이스터의 실험실 (마스터) 입장권 New 1 it

노블레스 코드 (익스퍼트) 입장권 Error ->  HTTPSConnectionPool(host='api.neople.co.kr', port=443): Max retries exceeded with url: /df/auction?itemName=%EB%85%B8%EB%B8%94%EB%A0%88%EC%8A%A4+%EC%BD%94%EB%93%9C+%28%EC%9D%B5%EC%8A%A4%ED%8D%BC%ED%8A%B8%29+%EC%9E%85%EC%9E%A5%EA%B6%8C&limit=100&sort=auctionNo%3Adesc&apikey=Sxwiysbra37brnJK9Fp12Z81vXvU0IEN (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fd5f6fd2a60>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))
노블레스 코드 : 더스크 (익스퍼트) 입장권 Error ->  HTTPSConnectionPool(host='api.neople.co.kr', port=443): Max retries exceeded with url: /df/auction?itemName=%EB%85%B8%EB%B8%94%EB%A0%88%EC%8A%A4+%EC%BD%94%EB%93%9C+%3A+%EB%8D%94%EC%8A%A4%ED%81%AC+%28%EC%9D%B5%EC%8A%A4%ED%8D%BC%ED%8A%B8%29+%EC%9E%85%EC%9E%A5%EA%B6%8C&limit=100&sort=auctionNo%3Adesc&apikey=Sxwiysbra37brnJK9Fp12Z81vXvU0IEN (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fd5f6fd25b0>: Failed

노블레스 코드 (마스터) 입장권 Error ->  HTTPSConnectionPool(host='api.neople.co.kr', port=443): Max retries exceeded with url: /df/auction?itemName=%EB%85%B8%EB%B8%94%EB%A0%88%EC%8A%A4+%EC%BD%94%EB%93%9C+%28%EB%A7%88%EC%8A%A4%ED%84%B0%29+%EC%9E%85%EC%9E%A5%EA%B6%8C&limit=100&sort=auctionNo%3Adesc&apikey=Sxwiysbra37brnJK9Fp12Z81vXvU0IEN (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fd5f6fd2850>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))
노블레스 코드 : 더스크 (마스터) 입장권 Error ->  HTTPSConnectionPool(host='api.neople.co.kr', port=443): Max retries exceeded with url: /df/auction?itemName=%EB%85%B8%EB%B8%94%EB%A0%88%EC%8A%A4+%EC%BD%94%EB%93%9C+%3A+%EB%8D%94%EC%8A%A4%ED%81%AC+%28%EB%A7%88%EC%8A%A4%ED%84%B0%29+%EC%9E%85%EC%9E%A5%EA%B6%8C&limit=100&sort=auctionNo%3Adesc&apikey=Sxwiysbra37brnJK9Fp12Z81vXvU0IEN (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fd5f6fd22b0>: Failed to establish a new 

regi 마이스터의 실험실 (노멀) 입장권 New 1 items
sold 노블레스 코드 (노멀) 입장권 New 1 items
regi 노블레스 코드 : 더스크 (익스퍼트) 입장권 New 1 items
sold 마이스터의 실험실 (마스터) 입장권 New 1 items
sold 노블레스 코드 (마스터) 입장권 New 1 items
regi 마이스터의 실험실 (익스퍼트) 입장권 New 1 items
regi 노블레스 코드 (익스퍼트) 입장권 New 2 items
sold 마이스터의 실험실 (마스터) 입장권 New 9 items
regi 마이스터의 실험실 (마스터) 입장권 New 3 items
sold 마이스터의 실험실 (마스터) 입장권 New 2 items
regi 노블레스 코드 : 더스크 (마스터) 입장권 New 1 items
regi 파괴된 죽은 자의 성 (노멀) 입장권 New 1 items
sold 노블레스 코드 (노멀) 입장권 New 1 items
regi 노블레스 코드 : 더스크 (익스퍼트) 입장권 New 1 items
regi 마이스터의 실험실 (마스터) 입장권 New 1 items
regi 노블레스 코드 : 더스크 (마스터) 입장권 New 1 items
regi 파괴된 죽은 자의 성 (노멀) 입장권 New 1 items
sold 마이스터의 실험실 (익스퍼트) 입장권 New 2 items
sold 노블레스 코드 : 더스크 (익스퍼트) 입장권 New 2 items
regi 마이스터의 실험실 (마스터) 입장권 New 3 items
sold 마이스터의 실험실 (마스터) 입장권 New 1 items
regi 노블레스 코드 : 더스크 (마스터) 입장권 New 1 items
sold 파괴된 죽은 자의 성 (노멀) 입장권 New 1 items
sold 파괴된 죽은 자의 성 (익스퍼트) 입장권 New 1 items
regi 마이스터의 실험실 (마스터) 입장권 New 2 items
sold 노블레스 코드 (마스터) 입장권 New 1 items
regi 노블레스 코드 : 